In [1]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

dls = TextDataLoaders.from_folder(path, valid='test')
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
2,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg
3,"xxbos xxmaj the freedom of having your own xxmaj sea xxmaj going xxmaj power xxmaj boat , the excitement of going on underwater adventures a rugged , an 's man of an adventurer and xxunk so well endowed ! ) assistants in fine xxmaj bikinis were all definite selling points for "" sea xxup xxunk - 61 ) . \n\n xxmaj just what was the reason for producing a sort of sea going "" gun for hire "" * series . xxmaj let 's look closely now . xxmaj there must be a some clues around . \n\n xxmaj if we were to look back just a little , we see the xxup rko xxmaj radio xxmaj pictures production of xxup underwater ! ( 1955 ) . xxmaj it starred xxmaj jane xxmaj russell , xxmaj gilbert xxmaj roland , xxmaj richard xxmaj egan and xxmaj lori xxmaj nelson as",pos
4,"xxbos i felt duty bound to watch the 1983 xxmaj timothy xxmaj dalton / xxmaj zelah xxmaj clarke adaptation of "" jane xxmaj eyre , "" because xxmaj i 'd just written an article about the 2006 xxup bbc "" jane xxmaj eyre "" for xxunk . \n\n xxmaj so , i approached watching this the way xxmaj i 'd approach doing homework . \n\n i was irritated at first . xxmaj the lighting in this version is bad . xxmaj everyone / everything is washed out in a bright white xxunk light that , in some scenes , casts shadows on the wall behind the characters . \n\n xxmaj and the sound is poorly recorded . i felt like i was listening to a high school play . \n\n xxmaj and the pancake make - up is way too heavy . \n\n xxmaj and the sets do n't fully",pos
5,"xxbos xxmaj god ! xxmaj zorro has been the the subject of about as many movies as xxmaj tarzan , and probably had about as many actors in the title role . \n\n xxmaj this xxmaj serial is one of my own per

In [2]:
learn = text_classifier_learner(dls,AWD_LSTM,drop_mult=0.5,metrics=accuracy)

In [3]:
learn.fine_tune(4,1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.443833,0.396215,0.818040,03:29


epoch,train_loss,valid_loss,accuracy,time
0,0.302000,0.286773,0.880120,07:00
1,0.233778,0.231721,0.909000,07:01
2,0.193860,0.189765,0.928000,07:01
3,0.145928,0.186722,0.930800,07:06


In [3]:
learn.show_results()

In [4]:
learn.predict("That movie was wicked cool !")

('pos', tensor(1), tensor([0.4265, 0.5735]))

In [6]:
# 在IMDb上微调语言模型
dls_lm = TextDataLoaders.from_folder(path,is_lm=True,vaild_pct=0.1)
dls_lm.show_batch(max_n=5)
learn1 = language_model_learner(dls_lm,AWD_LSTM,metrics=[accuracy,Perplexity()],path=path,wd=0.1).to_fp16()
learn1.fit_one_cycle(1,1e-2)

,text,text_
0,"xxbos xxmaj no , xxmaj i 've never seen any of the "" santa xxmaj slasher "" series , i.e. ' silent xxmaj night , xxmaj deadly xxmaj night , ' the original ' black xxmaj christmas ' or this one , ' christmas xxmaj evil . ' xxmaj i 've heard all about their reputation , or , xxup mads ( mothers xxmaj against xxmaj deranged xxmaj santas . ) i","xxmaj no , xxmaj i 've never seen any of the "" santa xxmaj slasher "" series , i.e. ' silent xxmaj night , xxmaj deadly xxmaj night , ' the original ' black xxmaj christmas ' or this one , ' christmas xxmaj evil . ' xxmaj i 've heard all about their reputation , or , xxup mads ( mothers xxmaj against xxmaj deranged xxmaj santas . ) i thought"
1,xxmaj memphis is known as one of the most notorious car thieves in xxmaj los xxmaj angeles . xxmaj memphis heads to a junkyard and meets xxmaj raymond xxunk xxmaj xxunk ) . xxmaj this guy threatens to kill xxmaj kip if xxmaj memphis does n't deliver 50 cars within 72 hours . \n\n xxmaj there are a few problems with this film : \n\n xxunk : xxmaj the first 48 in,memphis is known as one of the most notorious car thieves in xxmaj los xxmaj angeles . xxmaj memphis heads to a junkyard and meets xxmaj raymond xxunk xxmaj xxunk ) . xxmaj this guy threatens to kill xxmaj kip if xxmaj memphis does n't deliver 50 cars within 72 hours . \n\n xxmaj there are a few problems with this film : \n\n xxunk : xxmaj the first 48 in -
2,"\n\n xxmaj music : 10 / 10 xxmaj it does n't always work when using music sometimes it just does n't fit but that is not the thing in this movie , the music is perfect in tune , it makes the movie even more compelling . xxmaj this part of the movie will shine off as good as the other parts , a great soundtrack for a xxmaj romance / xxmaj","xxmaj music : 10 / 10 xxmaj it does n't always work when using music sometimes it just does n't fit but that is not the thing in this movie , the music is perfect in tune , it makes the movie even more compelling . xxmaj this part of the movie will shine off as good as the other parts , a great soundtrack for a xxmaj romance / xxmaj comedy"
3,"snatchers and xxmaj the faculty . \n\n xxmaj if you ever have the misfortune to come across this movie , i suggest you burn it . \n\n xxmaj how it classes itself as a horror movie when there was no use of fear , suspense , gore , terror or any of the usual horror film techniques i do n't know , however if you still find yourself wanting to watch this","and xxmaj the faculty . \n\n xxmaj if you ever have the misfortune to come across this movie , i suggest you burn it . \n\n xxmaj how it classes itself as a horror movie when there was no use of fear , suspense , gore , terror or any of the usual horror film techniques i do n't know , however if you still find yourself wanting to watch this film"
4,"n't much to do in this pre - celebrity performance . xxmaj highlight : the real signs of toddler temper when a few of the infants unexpectedly get well & truly soaked with milk . \n\n xxmaj often overlooked or neglected today , the one and two - reel short subjects were useful to the xxmaj studios as important training grounds for new or burgeoning talents , both in front & behind","much to do in this pre - celebrity performance . xxmaj highlight : the real signs of toddler temper when a few of the infants unexpectedly get well & truly soaked with milk . \n\n xxmaj often overlooked or neglected today , the one and two - reel short subjects were useful to the xxmaj studios as important training grounds for new or burgeoning talents , both in front & behind the"


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.390593,None,None,None,10:01


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [7]:
learn1.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [11]:
# 模型最后一层将激活值转换为词表中每个单词的概率。除去模型最后一层，称之为编码器。
learn1 = learn1.load('1epoch')
learn1.unfreeze()
learn1.fit_one_cycle(10,1e-3)
learn1.save_encoder('finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.153458,None,None,None,10:31
1,4.091826,None,None,None,10:20
2,4.045990,None,None,None,10:41


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")
/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")
/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


KeyboardInterrupt: ignored

## 编码器
编码器是模型的一部分，将输入文本映射成一些向量，通常用来解释具体的NLP任务。编码器本身并不特定于具体任务，通常与“编码器”或“头部”一起使用，后者用于将向量映射到某些特定任务设计的数据结构。

In [12]:
# 使用该模型生成随机评论
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn1.predict(TEXT,N_WORDS,temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

i liked this movie because i was not expecting a really good movie , so i wanted to watch it for the first time but do n't even expect any new language , but i found it to be excellent . It was a
i liked this movie because it was so good i wanted to go see it . But I 'm so addicted to Shakespeare and this movie was so good . My favorite part was with the Scorsese , which is


In [13]:
dls_clas=TextDataLoaders.from_folder(untar_data(URLs.IMDB),vaild='text',text_vocab=dls_lm.vocab)
learn2=text_classifier_learner(dls,AWD_LSTM,drop_mult=0.5,metrics=accuracy)
learn2=learn2.load_encoder('finetuned')


FileNotFoundError: ignored

In [ ]:
# 采用差异化学习率，逐步解冻
learn2.fit_one_cycle(1,2e-2)


In [ ]:
learn2.freeze_to(-2)
learn2.fit_one_cycle(1,slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn2.freeze_to(-3)
learn2.fit_one_cycle(1,slice(5e-2/(2.6**4),5e-3))

In [ ]:
# 解冻整个模型
learn2.unfreeze()
learn2.fit_one_cycle(2,slice(1e-3/(2.6**4),1e-3))

## 使用Hugging Face系列库进行推理

In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install transformers
import torch
from transformers import GPT2Tokenizer,GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "With great power comes great "
indexed_tokens = tokenizer.encode(text)

tokens_tensor = torch.tensor([indexed_tokens])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
print(tokens_tensor)
print(indexed_tokens)

tensor([[3152, 1049, 1176, 2058, 1049,  220]])
[3152, 1049, 1176, 2058, 1049, 220]


In [20]:
# 加载模型权重
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.eval()

with torch.no_grad():
  outputs= model(tokens_tensor)
  predictions = outputs[0]

predicted_index = torch.argmax(predictions[0,-1,:]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

print(predicted_text)

With great power comes great ills


In [21]:
len(outputs)

2

In [22]:
outputs[0].shape

torch.Size([1, 6, 50257])

In [27]:
print(predictions)

tensor([[[ -32.9691,  -32.2417,  -35.0687,  ...,  -37.8427,  -38.3769,
           -32.6595],
         [ -88.2333,  -87.4561,  -94.1138,  ...,  -91.9931,  -95.2675,
           -90.7904],
         [ -62.1415,  -64.1625,  -72.3117,  ...,  -72.0955,  -71.1991,
           -66.7285],
         [ -94.2255,  -94.3511, -101.4683,  ..., -100.7295, -100.7159,
           -97.0163],
         [ -99.9127,  -99.7470, -106.7008,  ..., -106.1145, -109.8075,
          -102.8086],
         [ -57.2461,  -60.1474,  -63.3509,  ...,  -72.5494,  -69.3108,
           -61.1931]]])


In [26]:
print(predictions[0,-1,:])

tensor([-57.2461, -60.1474, -63.3509,  ..., -72.5494, -69.3108, -61.1931])


In [31]:
print(predictions[0,-1,:])

tensor([-57.2461, -60.1474, -63.3509,  ..., -72.5494, -69.3108, -61.1931])


In [32]:
torch.argmax(predictions[0,-1,:])

tensor(2171)

In [33]:
torch.argmax(predictions[0,-1,:]).item()

2171

In [34]:
tokenizer.decode([predicted_index])

'ills'

In [35]:
torch.argmax(predictions[0,1,:]).item()

1943

In [42]:
tokenizer.decode(torch.argmax(predictions[0,-1,:]).item())

'ills'

In [38]:
print(predicted_text)

With great power comes great ills
